**Notações:**

-O arquivo csv foi extraído apartir do endereço http://mensario.portodesantos.com.br/cargas/ com os filtros marcados em Navegação:(TODAS AS NAVEGAÇÕES) e Tipo de Instalação:(TODAS AS INSTALAÇÕES).

-Esse processo de ETL se baseia no repositório ao público do github de Renato Campos (renato-campos) https://github.com/renato-campos/pi3-porto-de-santos.git

-Aqui o objetivo foi apenas extrair uma série temporal dos dados do porto de Santos.

**Importação das bibliotecas**

In [1]:
import pandas as pd

**Lendo o arquivo CSV**

In [2]:
df = pd.read_csv('dados/exportacao_cargas.csv')

**Investigando as Colunas, removendo e limpando df**

In [3]:
df.columns

Index(['TOTAL_TEU', 'NUMERO_VIAGEM', 'TIPO_NAVEGACAO', 'MOVIMENTO', 'ANO',
       'TERMINAIS', 'TOTAL_TONELADAS', 'NATUREZA_CARGA', 'TOTAL_UNID', 'MES',
       'CLASSENAVIO', 'SENTIDO', 'TIPO_INSTALACAO', 'BERCOS', 'MERCADORIAS',
       'METRICA', 'ANO_MES'],
      dtype='object')

In [4]:
removendo_colunas = ['TIPO_INSTALACAO','TERMINAIS','TOTAL_TEU','TOTAL_UNID','METRICA','ANO_MES','NUMERO_VIAGEM','CLASSENAVIO','MOVIMENTO']
clean_df = df.drop(removendo_colunas, axis=1)
clean_df.columns

Index(['TIPO_NAVEGACAO', 'ANO', 'TOTAL_TONELADAS', 'NATUREZA_CARGA', 'MES',
       'SENTIDO', 'BERCOS', 'MERCADORIAS'],
      dtype='object')

In [5]:
renomeando_colunas = {'BERCOS':'bercos','MERCADORIAS':'carga','ANO':'ano','TIPO_NAVEGACAO':'navegacao','TOTAL_TONELADAS':'toneladas','NATUREZA_CARGA':'perfil','MES':'mes','SENTIDO':'sentido'}
clean_df = clean_df.rename(columns=renomeando_colunas)
clean_df.columns

Index(['navegacao', 'ano', 'toneladas', 'perfil', 'mes', 'sentido', 'bercos',
       'carga'],
      dtype='object')

In [6]:
clean_df.head()

,navegacao,ano,toneladas,perfil,mes,sentido,bercos,carga
0,LONGO CURSO,2013,2210.139,CARGA CONTEINERIZADA,5,EMBARQUE,ECOP 2,OUTRAS MERCADORIAS
1,LONGO CURSO,2013,244.590,CARGA CONTEINERIZADA,6,DESEMBARQUE,SBR 1,OUTRAS MERCADORIAS
2,LONGO CURSO,2013,255.179,CARGA CONTEINERIZADA,7,DESEMBARQUE,SBR 1,PRODUTOS DIVERSOS DA INDÚSTRIA QUÍMICA
3,LONGO CURSO,2013,33.410,CARGA CONTEINERIZADA,7,DESEMBARQUE,SBR 1,SEM CARGAS
4,LONGO CURSO,2013,1171.472,CARGA CONTEINERIZADA,6,EMBARQUE,SBR 2,SEM CARGAS


**Selecionando período para série temporal**

In [ ]:
inicio = 2019
final = 2023
clean_df = clean_df[(clean_df['ano'] >= inicio) & (clean_df['ano'] <= final)]
clean_df = clean_df.sort_values(['ano', 'mes', 'navegacao', 'perfil', 'sentido', 'carga', 'bercos'])
clean_df.head()

,navegacao,ano,toneladas,perfil,mes,sentido,bercos,carga
265228,CABOTAGEM,2019,57.718,CARGA CONTEINERIZADA,1,DESEMBARQUE,SBR 2,ANIMAIS VIVOS
385791,CABOTAGEM,2019,96.930,CARGA CONTEINERIZADA,1,DESEMBARQUE,SBR 2,ANIMAIS VIVOS
357347,CABOTAGEM,2019,110.560,CARGA CONTEINERIZADA,1,DESEMBARQUE,SBR 3,ANIMAIS VIVOS
663736,CABOTAGEM,2019,46.040,CARGA CONTEINERIZADA,1,DESEMBARQUE,SBR 1,AÇÚCAR
451594,CABOTAGEM,2019,256.380,CARGA CONTEINERIZADA,1,DESEMBARQUE,SBR 3,AÇÚCAR


**Agregando os registros e valores e Convertendo valores das Toneladas**

In [8]:
clean_df = clean_df.groupby(['ano', 'mes', 'perfil', 'navegacao', 'sentido', 'carga', 'bercos']).sum().reset_index()
clean_df['toneladas'] = clean_df['toneladas'].round(0)
clean_df['toneladas'] = clean_df['toneladas'].astype('int')
clean_df.head(100)

,ano,mes,perfil,navegacao,sentido,carga,bercos,toneladas
0,2019,1,CARGA CONTEINERIZADA,CABOTAGEM,DESEMBARQUE,ANIMAIS VIVOS,SBR 2,155
1,2019,1,CARGA CONTEINERIZADA,CABOTAGEM,DESEMBARQUE,ANIMAIS VIVOS,SBR 3,111
2,2019,1,CARGA CONTEINERIZADA,CABOTAGEM,DESEMBARQUE,AÇÚCAR,SBR 1,46
3,2019,1,CARGA CONTEINERIZADA,CABOTAGEM,DESEMBARQUE,AÇÚCAR,SBR 3,2144
4,2019,1,CARGA CONTEINERIZADA,CABOTAGEM,DESEMBARQUE,CAFÉ,BTP 01,1570
...,...,...,...,...,...,...,...,...
95,2019,1,CARGA CONTEINERIZADA,CABOTAGEM,EMBARQUE,ACESSÓRIOS DE VEÍCULOS AUTOMÓVEIS,DPW 2,5
96,2019,1,CARGA CONTEINERIZADA,CABOTAGEM,EMBARQUE,ALGODÃO,BTP 01,192
97,2019,1,CARGA CONTEINERIZADA,CABOTAGEM,EMBARQUE,ALGODÃO,BTP 03,31
98,2019,1,CARGA CONTEINERIZADA,CABOTAGEM,EMBARQUE,ALGODÃO,SBR 2,22


In [9]:
serietemporal = clean_df.groupby(['ano', 'mes'])['toneladas'].sum().reset_index()
serietemporal = serietemporal.rename(columns={'toneladas':'porto'})
serietemporal.head()

,ano,mes,porto
0,2019,1,8982718
1,2019,2,10069129
2,2019,3,11384405
3,2019,4,10622636
4,2019,5,10993739


**Criando dicionário de cargas**

In [10]:
carga_dict = {
    'CARNES' : 'carnes',
    'OUTRAS MERCADORIAS' : 'outras',
    'PRODUTOS QUÍMICOS' : 'prod_quimicos',
    'SAL' : 'sal',
    'SEM CARGAS' : 'sem_cargas',
    'VEÍCULOS' : 'veiculos',
    'CELULOSE' : 'celulose',
    'SOJA EM GRÃOS' : 'graos',
    'PETRÓLEO E DERIVADOS' : 'petroleo',
    'PRODUTOS DA INDÚSTRIA QUÍMICA' : 'ind_quimica',
    'AÇÚCAR' : 'sugar',
    'CARVÃO' : 'carvao',
    'COMBUSTÍVEIS, ÓLEOS E PRODUTOS M' : 'combustiveis',
    'ETANOL' : 'etanol',
    'FERTILIZANTES' : 'fertiliz',
    'GÁS DE PETRÓLEO' : 'gas',
    'MINÉRIO DE FERRO' : 'ferro',
    'SUCOS' : 'sucos',
    'ÓLEO DE ORIGEM VEGETAL' : 'ol_beg',
    'PARTES/MOTORES/GERADORES/PÁS EÓL': 'mot_ger',
    'BUNKER (DIESEL)' : 'bunk_dies',
    'BUNKER (O.COMBUSTIVEL)': 'bunk_comb',
    'ENXOFRE' : 'enxofre',
    'BIODIESEL' : 'biodiesel'
}

**Filtrando as respectivas Cargas**

In [12]:
valor_carga = ['SOJA EM GRÃOS', 'OUTRAS MERCADORIAS', 'AÇÚCAR']

for k in valor_carga:
    carga_df = clean_df.loc[clean_df['carga'] == k]
    carga_df = carga_df.groupby(['ano', 'mes'])['toneladas'].sum().reset_index()
    carga_df = carga_df.rename(columns={'toneladas': carga_dict[k]})
    
    serietemporal = serietemporal.merge(carga_df, on=['ano', 'mes'], how='left')

**Nesse se vermos Jan/2021 está como NaN, mas em pesquisa ao mensário estatísitco constava que:**

"❖ MOVIMENTO MENSAL
O Porto de Santos iniciou 2021 registrando a movimentação de 9,2 milhões de toneladas neste mês de janeiro (10,6% ante janeiro de 2020). Trata-se do melhor
desempenho para janeiro em toda a nossa série histórica apresentar resultado 1,5% superior ao verificado no mesmo mês do ano de 2019.
Esse resultado reflete a alta de 9,0% nos embarques, que totalizaram 5,7 milhões de t em 2021. As maiores variações absolutas positivas foram observadas nos
embarques de açúcar a granel, +391,3 mil t (47,5%); farelo de soja a granel, +150,3 mil t (56,9%) e carga conteinerizada, +132,8 mil t (6,8%). Já as reduções mais expressivas
ocorreram nos embarques de soja em grãos a granel, -478,6 mil t (-100,0%)"

Logo preencheremos tal data com 0

In [15]:
serietemporal['graos'] = serietemporal['graos'].fillna(0)

**Salvando arquivo**

In [16]:
serietemporal.to_csv('outputs/serietemporal.csv', index=False)